**# 图像卷积**

**1. 互相关运算**

In [ ]:
!pip install d2l
!pip install matplotlib==3.0

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K): 
    """计算二维互相关运算"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()   
    return Y

In [ ]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

**2. 实现二维卷积层**

In [ ]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

**3. 图像中目标的边缘检测**

* 如下是卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。

In [ ]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [ ]:
K = torch.tensor([[1.0, -1.0]])  # 构造卷积核

* 输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [ ]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

* 卷积核K只能检测垂直边缘

In [ ]:
corr2d(X.t(), K)  # 将X转置之后，就无效了

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

**4. 学习生成卷积核**

* 先构造一个卷积层，并将其卷积核初始化为随机张量。在每次迭代中，比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。

* 在此使用内置的二维卷积层，并忽略偏置。

In [ ]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(20):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.5f}')

epoch 2, loss 17.78815
epoch 4, loss 6.19293
epoch 6, loss 2.35323
epoch 8, loss 0.93312
epoch 10, loss 0.37704
epoch 12, loss 0.15357
epoch 14, loss 0.06276
epoch 16, loss 0.02568
epoch 18, loss 0.01051
epoch 20, loss 0.00431


In [ ]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9931, -1.0066]])